![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/17.Graph_builder_for_DL_models.ipynb)

In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

license_keys.keys()

In [ ]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

In [ ]:
from sparknlp_jsl.training import tf_graph
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
tf_graph.get_models()


['ner_dl', 'generic_classifier', 'assertion_dl', 'relation_extraction']

## NER_DL

In [ ]:
tf_graph.print_model_params("ner_dl")


ner_dl parameters.
Parameter            Required   Default value        Description
ntags                yes        -                    Number of tags.
embeddings_dim       no         200                  Embeddings dimension.
nchars               no         100                  Number of chars.
lstm_size            no         128                  Number of LSTM units.
gpu_device           no         0                    Device for training.
is_medical           no         0                    Build a Medical Ner graph.


In [ ]:
tf_graph.build("ner_dl",build_params={"embeddings_dim": 200, "nchars": 80, "ntags": 12, "is_medical": 1}, model_location="./medical_ner_graphs", model_filename="auto")

Instructions for updating:
non-resource variables are not supported in the long term

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for upda

## AssertionDL

In [ ]:
tf_graph.print_model_params("assertion_dl")


assertion_dl parameters.
Parameter            Required   Default value        Description
max_seq_len          no         250                  Maximum sequence length.
feat_size            no         200                  Feature size.
n_classes            yes        -                    Number of classes.
device               no         /cpu:0               Device for training.
n_hidden             no         34                   Number of hidden units.


In [ ]:
tf_graph.build("assertion_dl",build_params={"n_classes": 10}, model_location="/tmp", model_filename="auto")





Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API





assertion_dl graph exported to /tmp/blstm_34_32_30_200_10.pb


## GenericClassifier

In [ ]:
tf_graph.print_model_params("generic_classifier")


generic_classifier parameters.
Parameter            Required   Default value        Description
hidden_layers        no         [200]                List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim            yes        -                    Input dimension.
output_dim           yes        -                    Output dimension.
hidden_act           no         relu                 Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2        no         0                    L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2    no         0                    L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm           no         0                    Batch normalization. Boolean (0 or 1).


In [ ]:
tf_graph.build("generic_classifier",build_params={"input_dim": 100, "output_dim": 10, "hidden_layers": [300, 200, 100], "hidden_act": "tanh"}, model_location="/tmp", model_filename="auto")

generic_classifier graph exported to /tmp/gcl.100.10.pb


## RelationExtraction

In [ ]:
tf_graph.print_model_params("relation_extraction")


relation_extraction parameters.
Parameter            Required   Default value        Description
hidden_layers        no         [200]                List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim            yes        -                    Input dimension.
output_dim           yes        -                    Output dimension.
hidden_act           no         relu                 Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2        no         0                    L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2    no         0                    L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm           no         0                    Batch normalization. Boolean (0 or 1).


In [ ]:

tf_graph.build("relation_extraction",build_params={"input_dim": 6000, "output_dim": 3, 'batch_norm':1, "hidden_layers": [300, 200], "hidden_act": "relu", 'hidden_act_l2':1}, model_location=".", model_filename="re_with_BN")

relation_extraction graph exported to ./re_with_BN
